In [1]:
from utils.hints import hide_toggle

# Набор данных "Auction Verification"

Датасет был создан в рамках научного исследования с целью выяснить, можно ли заменить дорогостоящую верификацию сложных моделей процессов (в данном случае: одновременные многораундовые аукционы, используемые для аукционов частотных спектров) предсказаниями результатов.

Каждый экземпляр в этом датасете представляет один запуск верификации. Верификация выполняется для проверки, является ли определенная цена возможной для определенного продукта, и в некоторых случаях, определения, может ли определенный участник выиграть продукт по этой цене. Каждый экземпляр содержит детали одного такого сценария верификации.

Краткое описание признаков:

- process.b1.capacity: Характеристика (вместимость) первого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 1.

- process.b2.capacity: Характеристика (вместимость) второго участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 2.

- process.b3.capacity: Характеристика (вместимость) третьего участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 3.

- process.b4.capacity: Характеристика (вместимость) четвертого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 4.

- property.price: Характеристика (цена) продукта. Целое число. Текущая проверенная цена.

- property.product: Характеристика (продукт) продукта. Целое число. Текущий проверенный продукт.

- property.winner: Характеристика (победитель) продукта. Целое число. Участник (заявитель), который в настоящее время признан победителем продукта (0, если проверена только цена).

Целевая переменная:

- verification.result: Целевой атрибут (категориальный). Результат верификации. Бинарный результат - возможен ли проверенный результат?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  # Импорт модели RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных
df = pd.read_csv("../Module2/data/auction.csv")
df.drop(columns=['verification.time'], inplace=True)

# # Определение признаков (X) и целевой переменной (y)
X = df.drop(columns=['verification.result'])
y = df['verification.result'].astype(int)

# Нормализация признаков с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Создание модели RandomForestClassifier
random_forest = RandomForestClassifier()

# Определение сетки параметров для перебора (n_estimators и max_depth)
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(random_forest, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(X_test)

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)


Наилучшие параметры на кросс-валидации: {'max_depth': 30, 'n_estimators': 300}
Наилучшая оценка на кросс-валидации: 0.9798071330744265
f1 на тестовом наборе:  0.9622641509433962


# Набор данных "Predict students' dropout and academic success"

Датасет был создан в рамках проекта, направленного на снижение количества студентов, прекращающих обучение или испытывающих неудачи в высшем образовании, путем использования методов машинного обучения для выявления студентов, находящихся в рискованных ситуациях на раннем этапе их учебного пути, чтобы разработать стратегии для их поддержки.

Датасет включает информацию, известную на момент поступления студента в учебное заведение, такую как учебный путь, демографические данные и социоэкономические факторы.

Задача сформулирована как задача классификации с тремя категориями (бросившие обучение, зачисленные и выпускники) к концу нормативного срока обучения. То есть, для каждого студента предсказывается, к какой из трех категорий он относится к моменту окончания курса (нормативного срока обучения).

Краткое описание признаков можно посмотреть по ссылке: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

Целевая переменная:

- **Target**: бросившие обучение, зачисленные и выпускники

In [4]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе Bike Sharing Demand. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных bike sharing
numeric_features = []
categorical_features = []
df = pd.read_csv("../Module2/data/students_predictions.csv", sep=";")
X = df.drop(["Target"], axis=1)
y = df["Target"]
for col in X.columns:
    if 2 < X[col].nunique() < 50:
        categorical_features.append(col)
    else:
        numeric_features.append(col)

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)
random_forest = RandomForestClassifier()

param_grid = {"n_estimators": [100, 200, 300], "max_depth": [None, 10, 20, 30]}

grid_search = GridSearchCV(random_forest, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление среднеквадратической ошибки (MSE) на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Наилучшие параметры на кросс-валидации: {'max_depth': None, 'n_estimators': 100}
Наилучшая оценка на кросс-валидации: 0.7719719671725043
f1 на тестовом наборе:  0.7867031300188788


/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [5, 6, 7, 9, 10, 11, 16, 17, 18, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [4]:
hide_toggle(for_next=True)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных bike sharing
numeric_features = []
categorical_features = []
df = pd.read_csv("datasets/students_predictions.csv", sep=";")

# Отделяем целевую переменную от признаков
X = df.drop(columns=['Target'])
y = df['Target']

for column in X.columns:
    unique_values_count = df[column].nunique()
    if 2 < unique_values_count < 50:
        categorical_features.append(column)
    else:
        numeric_features.append(column)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели RandomForestClassifier
random_forest = RandomForestClassifier()

# Определение сетки параметров для перебора (n_estimators и max_depth)
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(random_forest, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление среднеквадратической ошибки (MSE) на тестовом наборе
f1 = f1_score(y_pred, y_test, average='weighted')

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'max_depth': 30, 'n_estimators': 300}
Наилучшая оценка на кросс-валидации: 0.7747960268181782
f1 на тестовом наборе:  0.7862639325956047


C:\Users\avglinsky\PycharmProjects\math_and_ml_algorithms\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 6, 7, 9, 10, 11, 16, 17, 18, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Набор данных "accelerometer_gyro_mobile_phone_dataset"

Данные, собранные в 2022 году в Университете короля Сауда в Эр-Рияде для распознавания действий человека с помощью датчиков IMU мобильного телефона (акселерометр и гироскоп). эти действия сводятся к стоянию (остановке) или ходьбе.

Краткое описание признаков:
- accX	Вещественный
- accY	Вещественный
- accZ	Вещественный
- gyroX	Вещественный
- gyroY	Вещественный
- gyroZ	Вещественный
- timestamp	Datetime

Целевая переменная:

- **Activity**: стояние или ходьба

In [6]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе accelerometer_gyro_mobile_phone_dataset. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных Appliances Energy Prediction
df = pd.read_csv("../Module2/data/accelerometer_gyro_mobile_phone_dataset.csv")
df.head()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

,accX,accY,accZ,gyroX,gyroY,gyroZ,timestamp,Activity
0,-0.496517,3.785628,8.954828,-0.142849,-0.126159,-0.022539,34:22.9,1
1,-0.462388,3.869603,9.281898,0.084349,0.096695,0.092130,34:23.0,1
2,-0.296084,3.820505,8.930728,0.061763,0.051543,0.071287,34:23.1,1
3,-0.469723,3.890110,8.744067,0.007641,0.028679,0.109433,34:23.2,1
4,-0.472418,4.109105,8.941207,-0.123640,0.099057,0.051943,34:23.3,1


In [7]:
X = df.drop(["timestamp", "Activity"], axis=1)
y = df["Activity"]
numeric_features = X.columns.tolist()
numeric_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features)]
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features)]
)
# Создание модели RandomForestClassifier
random_forest = RandomForestClassifier()
param_grid = {"n_estimators": [100, 200, 300], "max_depth": [None, 10, 20, 30]}
grid_search = GridSearchCV(
    random_forest, param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [8]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе Bike Sharing Demand. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных bike sharing
numeric_features = []
categorical_features = []
df = pd.read_csv("../Module2/data/students_predictions.csv", sep=";")
X = df.drop(["Target"], axis=1)
y = df["Target"]
for col in X.columns:
    if 2 < X[col].nunique() < 50:
        categorical_features.append(col)
    else:
        numeric_features.append(col)

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)
random_forest = RandomForestClassifier()

param_grid = {"n_estimators": [100, 200, 300], "max_depth": [None, 10, 20, 30]}

grid_search = GridSearchCV(random_forest, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление среднеквадратической ошибки (MSE) на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Наилучшие параметры на кросс-валидации: {'max_depth': None, 'n_estimators': 300}
Наилучшая оценка на кросс-валидации: 0.7739493683024476
f1 на тестовом наборе:  0.7947563031086992


/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [5, 6, 7, 9, 10, 11, 16, 17, 18, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [7]:
hide_toggle(for_next=True)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных Appliances Energy Prediction
df = pd.read_csv("datasets/accelerometer_gyro_mobile_phone_dataset.csv")

# Удаление исходного столбца "date"
df = df.drop(columns=['timestamp'])

# Отделяем целевую переменную от признаков
X = df.drop(columns=['Activity'])
y = df['Activity']


# Предобработка данных: преобразование категориальных признаков и нормализация числовых признаков
numeric_features = X.columns.tolist() 

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])


# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели RandomForestClassifier
random_forest = RandomForestClassifier()

# Определение сетки параметров для перебора (n_estimators и max_depth)
param_grid = {'n_estimators': [10, 30, 59], 'max_depth': [None, 3, 5, 7]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(random_forest, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'max_depth': None, 'n_estimators': 59}
Наилучшая оценка на кросс-валидации: -0.016489457896971958
f1 на тестовом наборе:  0.9929738691087077
